# Plot all results -F1, std- found in a user specified folder


In [ ]:
import matplotlib.pyplot as plt
from displayutils import *
import numpy as np
import pandas as pd
from os.path import isfile, join
from os import listdir

#passive learning results
pl_results = dict()
pl_results['musicbrainz20K_mutated'] = 0.942
pl_results['musicbrainz20K'] = 0.966
pl_results['lspc_computers_mutated'] = 0.91
pl_results['lspc_computers'] = 0.95
pl_results['magellan_restaurants'] = 0.93

pl_results['musicbrainz20K_unbalanced'] = 0.944


pl_results_macro = dict()
pl_results_macro['musicbrainz20K_mutated'] = 0.942
pl_results_macro['musicbrainz20K'] = 0.962
pl_results_macro['lspc_computers_mutated'] = 0.891
pl_results_macro['lspc_computers'] = 0.967
pl_results_macro['magellan_restaurants'] = 0.939
pl_results_macro['musicbrainz20K_unbalanced'] = 0.945


In [ ]:
# Plot Results MICRO or MACRO F1
task = "magellan_restaurants"
results_folder = "../datasets/"+task+"/almser/plot/"
main_results_type = 'F1_micro'    
fig, ax = plt.subplots()

result_files = [f for f in sorted(listdir(results_folder)) if isfile(join(results_folder, f))]

handles = []
colors = ["#D55E00", "#2F73B2", "#4E9E73", "#000000",  "#CC79A7", "#F0E442"]
linestyles = ["-", "--", "-.", ":", "-", "--", "-."]

print(task)

if (len(result_files)>len(colors)): 
    print("You want to plot more results that provided colors and linestyles. Please add.")
for i in range(0,len(result_files)):
    if ('post_graph' in result_files[i] or 'exploit_explore' in result_files[i]  or 'almser_gb' in result_files[i] or 'GB_M_QHC' in result_files[i]): 
        results_type = 'F1_model_micro_boost_graph'
    else: results_type = main_results_type
    print(result_files[i].replace('.csv', ''))
    results = pd.read_csv(results_folder+result_files[i])
    #display(results.iloc[::10, : ])
    display(results.iloc[[74,124, 199], : ][[results_type, results_type+'_std',main_results_type,main_results_type+'_std']].round(3))
    #display(results.iloc[[99], : ][[results_type, results_type+'_std',main_results_type,main_results_type+'_std']].round(3))

    print("Difference to passive results at final iteration: %f" %(pl_results.get(task)-results.iloc[199][results_type]))

    query_num = np.arange(0,results.shape[0])
    label_name_parts = result_files[i].replace('.csv', '').split("_")
    label_name = ' '.join(label_name_parts[4:len(label_name_parts)-2])

    if label_name.startswith('almser') : label_name='ALMSER'
    elif label_name.startswith('disagreement') : label_name='B1: QHC'
    elif label_name.startswith('uncertainty') : label_name='B2: MB'
    
    f1_plt, = ax.plot(results[results_type], color=colors[i], linestyle=linestyles[i], label=label_name)
    errorfill(query_num, results[results_type].values, results[results_type+'_std'].values, color=colors[i], linestyle=linestyles[i])
    handles.append(f1_plt)
#passive learning result
if 'micro' in main_results_type:
    passive_res = pl_results.get(task)

if 'macro' in main_results_type:
    passive_res = pl_results_macro.get(task)
pas = ax.hlines(passive_res,0,200, color=colors[i+1], linestyle=linestyles[i+1], label='PASSIVE')
handles.append(pas)
ax.grid(True)
gridlines = ax.get_xgridlines()
for line in gridlines:
    line.set_linestyle('-.')
ax.set_xlabel("Active Learning Iterations (# Labels)", fontsize=15)
#ax.set_ylabel(results_type, fontsize=15)
ax.set_ylabel("F1", fontsize=15)

plt.xticks(fontsize=15)
plt.yticks(np.arange(0.0, 1.0, step=0.02), fontsize=15)



ax.legend(handles=handles, fontsize=12, loc='lower right')
plt.axis([0,200,0.80,0.96])

plt.title("ALMSER %s" %task)
plt.show()

